In [1]:
!pip install -q langchain_community streamlit tiktoken langchain-groq chromadb langchain-core langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.2 MB/s eta 0:0

# Using the Langchain

In [2]:
# enviroment variables
import os

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [3]:
# import libraries

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Chroma
import streamlit as st

In [4]:
# load data
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [5]:
# split data
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [6]:
# embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma.from_documents(chunks,embeddings)

base_retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [7]:
# creating multi queries
llm = ChatGroq(model_name='llama-3.1-70b-versatile')

final_retriever = MultiQueryRetriever.from_llm(base_retriever,llm)

In [8]:
final_retriever

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7bef7e7ccbe0>, search_kwargs={'k': 5}), llm_chain=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7bef746fafe0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7bef746fa170>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))
| LineListOut

In [9]:
# RAG

template = """
Answer the following questions base only the provided context.
if you don't know the answer say you don't know.

{context}

question : {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = ({'context':final_retriever,'question':RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser())

In [10]:
chain.invoke('what is agent system? and explain its components.')

'An Agent System, specifically a Large Language Model (LLM)-powered autonomous agent system, is a type of system that utilizes LLM as its core controller or brain. This system is designed to function autonomously, making decisions and taking actions based on the inputs it receives.\n\nThe Agent System consists of several key components, which work together to enable the agent to perform complex tasks and solve problems. These components include:\n\n1. **Planning**: This component involves breaking down complex tasks into smaller, manageable subgoals and planning the necessary steps to achieve them. The agent uses techniques like Chain of Thought (CoT) to think step-by-step and decompose hard tasks into simpler ones.\n\n2. **Memory**: The Agent System has two types of memory:\n   - **Short-term memory**: This refers to the in-context learning capabilities of the model, which allows it to learn and retain information over a short period.\n   - **Long-term memory**: This provides the agen

In [11]:
chain.invoke('explain the self reflection...')

"Self-reflection in the context of LLM-powered autonomous agents refers to the ability of the agent to improve iteratively by refining past action decisions and correcting previous mistakes. It is a crucial aspect of real-world tasks where trial and error are inevitable.\n\nSelf-reflection in LLM-powered agents can be achieved through various methods, including:\n\n1. Reflexion framework: This framework equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills. It uses a heuristic function to determine when the trajectory is inefficient or contains hallucination and should be stopped. The agent then uses two-shot examples to learn from its mistakes and improve its decision-making.\n\n2. Chain of Hindsight (CoH): This method encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback. The model is fine-tuned to predict the best output based on the feedback sequence, allo